In [1]:
import django_initializer
from ontrack.market.models.equity import EquityLiveOpenInterest
from ontrack.market.models.index import IndexLiveOpenInterest
import pandas as pd
import numpy as np
from ontrack.utils.datetime import DateTimeHelper as dt
import pytz
import matplotlib.pyplot as plt

In [2]:
def color_boolean(val):
    color =''
    foreground = ''
    if val == "BEARISH":
        color = 'red'
        foreground = 'black'        
    elif val == "BULLISH":
        color = 'green'        
        foreground = 'black'  
    return f'background-color: {color}; color: {foreground}'

In [4]:
start_d = dt.set_time_to_current_date(9, 0, 0, "Asia/Kolkata")
end_d = dt.current_date_time()
#records = IndexLiveOpenInterest.backend.filter(date__gte=d, entity__symbol__iexact="nifty").order_by("date")
records = EquityLiveOpenInterest.backend.filter(date__gte=start_d, date__lte=end_d).order_by("date")
df = pd.DataFrame(list(records.values("date", "entity__symbol", "lastest_open_interest", "previous_open_interest", "change_in_open_interest", "average_open_interest", "volume_open_interest", "underlying_value")))

common_names = {    
    "entity__symbol": "symbol",
    "lastest_open_interest": "latestOI",
    "previous_open_interest": "prevOI",
    "change_in_open_interest": "changeInOI",    
    "average_open_interest": "avgInOI",
    "volume_open_interest": "volume",
    "underlying_value": "underlyingvalue"
}
df.rename(columns=common_names, errors="ignore", inplace=True)
df = df.loc[df.groupby('symbol').date.idxmax()]

a = np.array(df['changeInOI'], dtype=float)
b = np.array(df['prevOI'], dtype=float)
df["change_per"] = np.divide(a, b, out=np.zeros_like(a), where= b!=0)
df["change_per"] = df["change_per"] * 100

df = df.set_index("date")
df.index = df.index.tz_convert('Asia/Kolkata')
df = df.sort_values('change_per', ascending=True)

print(df[["symbol", "prevOI", "latestOI", "change_per"]].head(10))
print(df[["symbol", "prevOI", "latestOI", "change_per"]].sort_values('change_per', ascending=False).head(10))

                               symbol      prevOI    latestOI  change_per
date                                                                     
2022-12-07 15:18:54+05:30         pnb  35890.0000  33402.0000   -6.932293
2022-12-07 15:18:54+05:30        gnfc  16153.0000  15335.0000   -5.064075
2022-12-07 15:18:54+05:30   whirlpool   3619.0000   3450.0000   -4.669798
2022-12-07 15:18:54+05:30     polycab   7950.0000   7667.0000   -3.559748
2022-12-07 15:18:54+05:30  bandhanbnk  59771.0000  58072.0000   -2.842516
2022-12-07 15:18:54+05:30     escorts  11191.0000  10949.0000   -2.162452
2022-12-07 15:18:54+05:30   hindpetro  14550.0000  14287.0000   -1.807560
2022-12-07 15:18:54+05:30  apollotyre  11771.0000  11594.0000   -1.503696
2022-12-07 15:18:54+05:30     mphasis  21726.0000  21436.0000   -1.334806
2022-12-07 15:18:54+05:30    shreecem  12402.0000  12238.0000   -1.322367
                              symbol      prevOI    latestOI  change_per
date                                   

In [5]:
d = dt.get_date_time(2022,12,6, 8, 5, 0)
#records = IndexLiveOpenInterest.backend.filter(date__gte=d, entity__symbol__iexact="nifty").order_by("date")
records = EquityLiveOpenInterest.backend.filter(date__gte=d, entity__symbol__iexact="siemens").order_by("date")
df = pd.DataFrame(list(records.values("date","lastest_open_interest", "previous_open_interest", "change_in_open_interest", "average_open_interest", "volume_open_interest", "underlying_value")))

common_names = {
    "lastest_open_interest": "latestOI",
    "previous_open_interest": "prevOI",
    "change_in_open_interest": "changeInOI",    
    "average_open_interest": "avgInOI",
    "volume_open_interest": "volume",
    "underlying_value": "underlyingvalue"
}
# df.set_index("date", inplace = True)
df.rename(columns=common_names, errors="ignore", inplace=True)

df = df.set_index("date")
df.index = df.index.tz_convert('Asia/Kolkata')
df['change_oi'] =  df["latestOI"].diff()
df['price_change'] =  df["underlyingvalue"].diff()

conditions = [
    (df["change_oi"] > 0) & (df['price_change'] > 0),
    (df["change_oi"] > 0) & (df['price_change'] < 0),
    (df["change_oi"] < 0) & (df['price_change'] > 0),
    (df["change_oi"] < 0) & (df['price_change'] < 0)]
choices = ["LONG BUILDUP", "SHORT BUILDUP", "SHORT COVERING", "LONG UNWINDING"]
df["position"] = np.select(conditions, choices)

choices = ["BULLISH", "BEARISH", "BULLISH", "BEARISH"]
df["direction"] = np.select(conditions, choices)

df1 = df.groupby('position').sum()
df_s = df.style.applymap(color_boolean, subset=['direction'])
df_s


,latestOI,prevOI,changeInOI,avgInOI,volume,underlyingvalue,change_oi,price_change,position,direction
date,,,,,,,,,,
2022-12-06 09:16:02+05:30,10491.0000,10491.0000,0.0000,0.0000,6.0000,2763.0000,nan,nan,0,0
2022-12-06 09:22:02+05:30,10498.0000,10491.0000,7.0000,0.0700,54.0000,2768.0000,7.0000,5.0000,LONG BUILDUP,BULLISH
2022-12-06 09:40:07+05:30,10510.0000,10491.0000,19.0000,0.1800,140.0000,2767.0000,12.0000,-1.0000,SHORT BUILDUP,BEARISH
2022-12-06 09:52:06+05:30,10523.0000,10491.0000,32.0000,0.3100,199.0000,2761.0000,13.0000,-6.0000,SHORT BUILDUP,BEARISH
2022-12-06 09:58:06+05:30,10500.0000,10491.0000,9.0000,0.0900,262.0000,2761.0000,-23.0000,0.0000,0,0
2022-12-06 10:03:08+05:30,10506.0000,10491.0000,15.0000,0.1400,267.0000,2761.0000,6.0000,0.0000,0,0
2022-12-06 10:09:06+05:30,10504.0000,10491.0000,13.0000,0.1200,322.0000,2760.0000,-2.0000,-1.0000,LONG UNWINDING,BEARISH
2022-12-06 10:13:07+05:30,10502.0000,10491.0000,11.0000,0.1000,344.0000,2762.0000,-2.0000,2.0000,SHORT COVERING,BULLISH
2022-12-06 10:19:07+05:30,10494.0000,10491.0000,3.0000,0.0300,371.0000,2758.0000,-8.0000,-4.0000,LONG UNWINDING,BEARISH


In [6]:
df1

df1["long"] = df1.loc["LONG BUILDUP"]["change_oi"] + df1.loc["LONG UNWINDING"]["change_oi"]
df1["short"] = df1.loc["SHORT BUILDUP"]["change_oi"] + df1.loc["SHORT COVERING"]["change_oi"]
df1["net"] = df1["long"] - df1["short"]

df1[["change_oi", "long", "short", "net"]]

,change_oi,long,short,net
position,,,,
0,190.0000,3532.0000,1719.0000,1813.0000
LONG BUILDUP,4407.0000,3532.0000,1719.0000,1813.0000
LONG UNWINDING,-875.0000,3532.0000,1719.0000,1813.0000
SHORT BUILDUP,2559.0000,3532.0000,1719.0000,1813.0000
SHORT COVERING,-840.0000,3532.0000,1719.0000,1813.0000


In [9]:
# TrueData Real Time API - Sample code
# for create_connectin you need to pip install websocket and pip install websocket-client

from websocket import create_connection

realtime_port = 8084  # 8082 for production & 8084 for sandbox environment during trial

username = 'FYERS1894'
password = 'gZDd0mxd'

print("\nCreating a connection with the server...\n")

ws = ''
try:
    ws = create_connection(f"wss://push.truedata.in:{realtime_port}?user={username}&password={password}")
    print("Sending 'Connection  Request'... for desired symbols ... ")
    print(ws.recv())
    print("Connection Established !\n")
    print("Adding Symbols, Now...\n")


    ws.send('{"method": "addsymbol", "symbols":["NIFTY 50","NIFTY BANK","MCXCOMPDEX","AARTIIND",'
            '"BRITANNIA","COLPAL","DMART","EICHERMOT","GILLETTE","HDFCBANK","ICICIBANK","JKTYRE","KAJARIACER",'
            '"LICHSGFIN","MINDTREE","OFSS","PNB","QUICKHEAL","RELIANCE","SBIN","TCS","UJJIVAN","WIPRO","YESBANK",'
            '"ZEEL","NIFTY-I","BANKNIFTY-I","UPL-I","VEDL-I",'
            '"VOLTAS-I","ZEEL-I","CRUDEOIL-I","GOLDM-I","SILVERM-I","COPPER-I", "SILVER-I"]}')


    while True:
        result = ws.recv()
        print("Received '%s'" % result)

except ConnectionError as error:
    print(error)
    ws.close()
    exit()

ImportError: cannot import name 'create_connection' from 'websocket' (c:\Sachin\OST-API\venv\lib\site-packages\websocket\__init__.py)